### Teste com auxilio do chatGPT

In [2]:
!pip install pandas

In [3]:
!pip install pandasql

  Using cached pandasql-0.7.3.tar.gz (26 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26800 sha256=1365635eed2e52e30664392feb86bd4fcc216b1ff9a4c98dd8a3745ae9db3fd3
  Stored in directory: c:\users\luanzera\appdata\local\pip\cache\wheels\4c\a4\ce\ad31f7d9f6bfce32741955de12e76e3a6e7181c20321db3c66
Successfully built pandasql


In [72]:
import pandas as pd
import sqlite3
from pandasql import sqldf

# Leitura dos arquivos excel
fotografia_Funded = pd.read_excel(r'C:\Users\luanzera\Desktop\Estudos\Taks BBA - Comparando arquivos\Arquivos p Comparar\Fotografia\Fotografia Operacoes Funded.xlsx')
Base_Unificada = pd.read_excel(r'C:\Users\luanzera\Desktop\Estudos\Taks BBA - Comparando arquivos\Arquivos p Comparar\BaseUnificada\BaseUnificada.xlsx')

# Dicionário com as colunas correspondentes
colunas = {
        'BANCO': 'BANCO',
        'PAIS_UNIDADE': 'Unidade',
        'BANK_CITY_LENDER': 'Bank/City Lender', 
        'CCY': 'CCY',
        'AMOUNT_RECEIVED': 'Amount',
        'AMOUNT_USD': 'Amount_USD', 
        'TRANSACTION': 'Transaction Type',
        'DEAL_DATE': 'Deal Date',
        'VALUE_DATE': 'Value Date', 
        'MATURITY': 'Maturity'
        }

# Renomeia as colunas do arquivo Fotografia
fotografia_Funded.rename(columns=colunas, inplace=True)

# Renomeando as colunas do arquivo A para as colunas do arquivo B
fotografia_Funded = fotografia_Funded.rename(columns=colunas)

# Criar conexão com o banco de dados em memória
conn = sqlite3.connect(':memory:')

# Definir a consulta SQL
with conn:
    # Criar a tabela fotografia_Funded
    fotografia_Funded.to_sql('fotografia_Funded', conn, if_exists='replace', index=False)
    # Adicionar a coluna Amount_USD
    # conn.execute("ALTER TABLE fotografia_Funded ADD COLUMN Amount_USD FLOAT;")
    # Criar a tabela Base_Unificada
    Base_Unificada.to_sql('Base_Unificada', conn, if_exists='replace', index=False)
    
    # Executar a consultas SQLs
    
    # Join utilizando SQL para comparar as colunas LEFT
    query1 = """
        SELECT *
        FROM fotografia_Funded
        LEFT OUTER JOIN Base_Unificada
        ON fotografia_Funded.BANCO = Base_Unificada.BANCO
        AND fotografia_Funded.Unidade = Base_Unificada.PAIS_UNIDADE
        AND fotografia_Funded.`Bank/City Lender` = Base_Unificada.BANK_CITY_LENDER
        AND fotografia_Funded.CCY = Base_Unificada.CCY
        AND fotografia_Funded.Amount = Base_Unificada.AMOUNT_RECEIVED
        AND fotografia_Funded.`Transaction Type` = Base_Unificada.`TRANSACTION`
        AND fotografia_Funded.`Deal Date` = Base_Unificada.DEAL_DATE
        AND fotografia_Funded.`Value Date` = Base_Unificada.VALUE_DATE
        AND fotografia_Funded.Maturity = Base_Unificada.MATURITY
        WHERE Base_Unificada.BANCO IS NULL
        """

    dados_left = pd.read_sql_query(query1, conn)
    

    # Join utilizando SQL para comparar as colunas LEFT
    query2 = """
            SELECT *
            FROM Base_Unificada
            LEFT OUTER JOIN fotografia_Funded
            ON fotografia_Funded.BANCO = Base_Unificada.BANCO
            AND fotografia_Funded.Unidade = Base_Unificada.PAIS_UNIDADE
            AND fotografia_Funded.`Bank/City Lender` = Base_Unificada.BANK_CITY_LENDER
            AND fotografia_Funded.CCY = Base_Unificada.CCY
            AND fotografia_Funded.Amount = Base_Unificada.AMOUNT_RECEIVED
            AND fotografia_Funded.`Transaction Type` = Base_Unificada.`TRANSACTION`
            AND fotografia_Funded.`Deal Date` = Base_Unificada.DEAL_DATE
            AND fotografia_Funded.`Value Date` = Base_Unificada.VALUE_DATE
            AND fotografia_Funded.Maturity = Base_Unificada.MATURITY
            WHERE fotografia_Funded.BANCO IS NULL
            """

    dados_right = pd.read_sql_query(query2, conn)

    # Join utilizando SQL para comparar as colunas INNER
    query3 = """
            SELECT *
            FROM Base_Unificada
            INNER JOIN fotografia_Funded
            ON fotografia_Funded.BANCO = Base_Unificada.BANCO
            AND fotografia_Funded.Unidade = Base_Unificada.PAIS_UNIDADE
            AND fotografia_Funded.`Bank/City Lender` = Base_Unificada.BANK_CITY_LENDER
            AND fotografia_Funded.CCY = Base_Unificada.CCY
            AND fotografia_Funded.Amount = Base_Unificada.AMOUNT_RECEIVED
            AND fotografia_Funded.`Transaction Type` = Base_Unificada.`TRANSACTION`
            AND fotografia_Funded.`Deal Date` = Base_Unificada.DEAL_DATE
            AND fotografia_Funded.`Value Date` = Base_Unificada.VALUE_DATE
            AND fotografia_Funded.Maturity = Base_Unificada.MATURITY
            """

    dados_comuns = pd.read_sql_query(query3, conn)


    # Join utilizando SQL para ver os incomuns entre as tabelas, FULL OUTER JOIN, NN CONCLUIDA
    query4 = """
            SELECT *
            FROM Base_Unificada
            FULL OUTER JOIN fotografia_Funded
            ON fotografia_Funded.BANCO = Base_Unificada.BANCO
            AND fotografia_Funded.Unidade = Base_Unificada.PAIS_UNIDADE
            AND fotografia_Funded.`Bank/City Lender` = Base_Unificada.BANK_CITY_LENDER
            AND fotografia_Funded.CCY = Base_Unificada.CCY
            AND fotografia_Funded.Amount = Base_Unificada.AMOUNT_RECEIVED
            AND fotografia_Funded.`Transaction Type` = Base_Unificada.`TRANSACTION`
            AND fotografia_Funded.`Deal Date` = Base_Unificada.DEAL_DATE
            AND fotografia_Funded.`Value Date` = Base_Unificada.VALUE_DATE
            AND fotografia_Funded.Maturity = Base_Unificada.MATURITY
            WHERE Base_Unificada.BANCO IS NULL OR fotografia_Funded.BANCO IS NULL;
            """
    dados_incomuns = pd.read_sql_query(query4, conn)
    
    


In [62]:
dados_left

,BANCO,Unidade,Bank/City Lender,CCY,Amount,Transaction,Transaction Type,Deal Date,Value Date,Maturity,...,AMOUNT_USD,AMOUNT_LC,TRANSACTION,DEAL_DATE,VALUE_DATE,MATURITY,ORIGINAL_MATURITY,TERM,PAIS_UNIDADE,HISTORY
0,JP MORGAN LUANZERA Foto,USA,JP MORGAN CHASE,USD,180000000,Importe Finance,TRADE,2023-12-06 00:00:00,14/03/2023,2023-12-06 00:00:00,...,None,None,None,None,None,None,None,None,None,None
1,SANTANDER,BRAZIL,SANTANDER,USD,56000000,Importe Finance,TRADE,2023-02-03 00:00:00,2023-02-05 00:00:00,2023-03-09 00:00:00,...,None,None,None,None,None,None,None,None,None,None
2,DALE DELE BANCO,BRAZIL,DALES BANK,USD,2342421,Importe Finance,TRADE,27/02/2023,2023-08-05 00:00:00,2023-01-08 00:00:00,...,None,None,None,None,None,None,None,None,None,None


In [63]:
dados_right

,AGE NUMBER,BANCO,BANK_CITY_LENDER,CCY,AMOUNT_RECEIVED,AMOUNT_USD,AMOUNT_LC,TRANSACTION,DEAL_DATE,VALUE_DATE,...,Bank/City Lender,CCY,Amount,Transaction,Transaction Type,Deal Date,Value Date,Maturity,Original Maturity,Term to Maturity
0,2,JP MORGAN da BASE,JP MORGAN CHASE,USD,180000000,180000000,None,TRADE,2023-12-06 00:00:00,14/03/2023,...,None,None,None,None,None,None,None,None,None,None
1,5,SANTANDER,SANTANDER,USD,56000000,56000000,None,TRADE,2023-02-03 00:00:00,2023-02-03 00:00:00,...,None,None,None,None,None,None,None,None,None,None
2,19,BANCO LUANZERA,LUANZERA BANK,USD,9824732,9824732,None,TRADE,2023-08-02 00:00:00,2023-12-05 00:00:00,...,None,None,None,None,None,None,None,None,None,None
3,20,BANCO NICOLAS,NICK BANK,USD,23948278,23948278,None,WK,2023-12-06 00:00:00,14/03/2023,...,None,None,None,None,None,None,None,None,None,None


In [64]:
dados_incomuns

,AGE NUMBER,BANCO,BANK_CITY_LENDER,CCY,AMOUNT_RECEIVED,AMOUNT_USD,AMOUNT_LC,TRANSACTION,DEAL_DATE,VALUE_DATE,...,Bank/City Lender,CCY,Amount,Transaction,Transaction Type,Deal Date,Value Date,Maturity,Original Maturity,Term to Maturity
0,2.0,JP MORGAN da BASE,JP MORGAN CHASE,USD,180000000.0,180000000.0,None,TRADE,2023-12-06 00:00:00,14/03/2023,...,None,None,NaN,None,None,None,None,None,None,None
1,5.0,SANTANDER,SANTANDER,USD,56000000.0,56000000.0,None,TRADE,2023-02-03 00:00:00,2023-02-03 00:00:00,...,None,None,NaN,None,None,None,None,None,None,None
2,19.0,BANCO LUANZERA,LUANZERA BANK,USD,9824732.0,9824732.0,None,TRADE,2023-08-02 00:00:00,2023-12-05 00:00:00,...,None,None,NaN,None,None,None,None,None,None,None
3,20.0,BANCO NICOLAS,NICK BANK,USD,23948278.0,23948278.0,None,WK,2023-12-06 00:00:00,14/03/2023,...,None,None,NaN,None,None,None,None,None,None,None
4,NaN,None,None,None,NaN,NaN,None,None,None,None,...,JP MORGAN CHASE,USD,180000000.0,Importe Finance,TRADE,2023-12-06 00:00:00,14/03/2023,2023-12-06 00:00:00,2023-12-06 00:00:00,260
5,NaN,None,None,None,NaN,NaN,None,None,None,None,...,SANTANDER,USD,56000000.0,Importe Finance,TRADE,2023-02-03 00:00:00,2023-02-05 00:00:00,2023-03-09 00:00:00,2023-03-09 00:00:00,360
6,NaN,None,None,None,NaN,NaN,None,None,None,None,...,DALES BANK,USD,2342421.0,Importe Finance,TRADE,27/02/2023,2023-08-05 00:00:00,2023-01-08 00:00:00,2023-01-08 00:00:00,1900-12-25 00:00:00


In [65]:
dados_comuns

,AGE NUMBER,BANCO,BANK_CITY_LENDER,CCY,AMOUNT_RECEIVED,AMOUNT_USD,AMOUNT_LC,TRANSACTION,DEAL_DATE,VALUE_DATE,...,Bank/City Lender,CCY,Amount,Transaction,Transaction Type,Deal Date,Value Date,Maturity,Original Maturity,Term to Maturity
0,1,BNY MELLON,BNY MELLON,USD,18000000,18000000,None,TRADE,2023-02-06 00:00:00,2023-10-03 00:00:00,...,BNY MELLON,USD,18000000,Importe Finance,TRADE,2023-02-06 00:00:00,2023-10-03 00:00:00,2023-02-06 00:00:00,2023-02-06 00:00:00,180
1,3,DEUTSCHE BANK,DEUTSCHE BANK,USD,27000000,27000000,None,TRADE,2023-06-03 00:00:00,2023-06-03 00:00:00,...,DEUTSCHE BANK,USD,27000000,Importe Finance,TRADE,2023-06-03 00:00:00,2023-06-03 00:00:00,2023-06-03 00:00:00,2023-06-03 00:00:00,260
2,4,HSBC,HSBC BRAZIL,USD,31000000,31000000,None,WK,28/02/2023,28/02/2023,...,HSBC BRAZIL,USD,31000000,Working Capital,WK,28/02/2023,28/02/2023,2023-12-05 00:00:00,2023-12-05 00:00:00,180
3,6,YES BANK,YES BANK,USD,22000000,22000000,None,WK,27/02/2023,28/02/2023,...,YES BANK,USD,22000000,Working Capital,WK,27/02/2023,28/02/2023,2023-01-08 00:00:00,2023-01-08 00:00:00,360
4,7,SOCIETE GENERALE,SOCIETE GENERALE,USD,87000000,87000000,None,TRADE,13/03/2023,2023-08-02 00:00:00,...,SOCIETE GENERALE,USD,87000000,Importe Finance,TRADE,13/03/2023,2023-08-02 00:00:00,2023-12-05 00:00:00,2023-12-05 00:00:00,260
5,8,JP MORGAN,JP MORGAN CHASE,USD,70000000,70000000,None,WK,20/03/2023,30/03/2023,...,JP MORGAN CHASE,USD,70000000,Working Capital,WK,20/03/2023,30/03/2023,2023-03-09 00:00:00,2023-03-09 00:00:00,180
6,9,SANTANDER,SANTANDER,USD,1000000,1000000,None,WK,15/03/2023,2023-12-05 00:00:00,...,SANTANDER,USD,1000000,Working Capital,WK,15/03/2023,2023-12-05 00:00:00,2023-01-08 00:00:00,2023-01-08 00:00:00,360
7,10,BANCOLOMBIA,BANCOLOMBIA,USD,27030000,27030000,None,STRUCT,2023-02-03 00:00:00,2023-03-09 00:00:00,...,BANCOLOMBIA,USD,27030000,Plegde Loan,STRUCT,2023-02-03 00:00:00,2023-03-09 00:00:00,2023-12-05 00:00:00,2023-12-05 00:00:00,90
8,11,BANCO DEL CHILE,BANCO DEL CHILE,USD,73000000,73000000,None,TRADE,22/08/23,13/11/23,...,BANCO DEL CHILE,USD,73000000,Importe Finance,TRADE,22/08/23,13/11/23,2023-03-09 00:00:00,2023-03-09 00:00:00,90
9,12,CHINA BANK,CHINA BANK,USD,64000000,64000000,None,TRADE,2023-01-08 00:00:00,26/12/23,...,CHINA BANK,USD,64000000,Importe Finance,TRADE,2023-01-08 00:00:00,26/12/23,2023-06-04 00:00:00,2023-06-04 00:00:00,180


In [ ]:
# Opcao para ver os dados comuns entre as tabelas
SELECT *
FROM Base_Unificada
LEFT JOIN fotografia_Funded
ON fotografia_Funded.BANCO = Base_Unificada.BANCO
AND fotografia_Funded.Unidade = Base_Unificada.PAIS_UNIDADE
AND fotografia_Funded.`Bank/City Lender` = Base_Unificada.BANK_CITY_LENDER
AND fotografia_Funded.CCY = Base_Unificada.CCY
AND fotografia_Funded.Amount = Base_Unificada.AMOUNT_RECEIVED
AND fotografia_Funded.`Transaction Type` = Base_Unificada.`TRANSACTION`
AND fotografia_Funded.`Deal Date` = Base_Unificada.DEAL_DATE
AND fotografia_Funded.`Value Date` = Base_Unificada.VALUE_DATE
AND fotografia_Funded.Maturity = Base_Unificada.MATURITY
WHERE fotografia_Funded.BANCO IS NULL

UNION

SELECT *
FROM Base_Unificada
RIGHT JOIN fotografia_Funded
ON fotografia_Funded.BANCO = Base_Unificada.BANCO
AND fotografia_Funded.Unidade = Base_Unificada.PAIS_UNIDADE
AND fotografia_Funded.`Bank/City Lender` = Base_Unificada.BANK_CITY_LENDER
AND fotografia_Funded.CCY = Base_Unificada.CCY
AND fotografia_Funded.Amount = Base_Unificada.AMOUNT_RECEIVED
AND fotografia_Funded.`Transaction Type` = Base_Unificada.`TRANSACTION`
AND fotografia_Funded.`Deal Date` = Base_Unificada.DEAL_DATE
AND fotografia_Funded.`Value Date` = Base_Unificada.VALUE_DATE
AND fotografia_Funded.Maturity = Base_Unificada.MATURITY
WHERE Base_Unificada.BANCO IS NULL


In [70]:
#ver as colunas, Foto
for col in dados_comuns.columns:
    print(col)

AGE NUMBER
BANCO
BANK_CITY_LENDER
CCY
AMOUNT_RECEIVED
AMOUNT_USD
AMOUNT_LC
TRANSACTION
DEAL_DATE
VALUE_DATE
MATURITY
ORIGINAL_MATURITY
TERM
PAIS_UNIDADE
HISTORY
BANCO
Unidade
Bank/City Lender
CCY
Amount
Transaction
Transaction Type
Deal Date
Value Date
Maturity
Original Maturity
Term to Maturity


In [73]:
#ver as colunas, Base
for colBase in dados_left.columns:
    print(colBase)

BANCO
Unidade
Bank/City Lender
CCY
Amount
Transaction
Transaction Type
Deal Date
Value Date
Maturity
Original Maturity
Term to Maturity
AGE NUMBER
BANCO
BANK_CITY_LENDER
CCY
AMOUNT_RECEIVED
AMOUNT_USD
AMOUNT_LC
TRANSACTION
DEAL_DATE
VALUE_DATE
MATURITY
ORIGINAL_MATURITY
TERM
PAIS_UNIDADE
HISTORY


In [ ]:

# Selecionando dados exclusivos do arquivo A
dados_left = pd.merge(fotografia_Funded, Base_Unificada, on=list(colunas.values()), how='left').query('coluna_b_1.isnull()', engine='python')

# Selecionando dados exclusivos do arquivo B
dados_right = pd.merge(fotografia_Funded, Base_Unificada, on=list(colunas.values()), how='right').query('coluna_a_1.isnull()', engine='python')

# Salva os dados em arquivos excel
with pd.ExcelWriter('Dados_comuns.xlsx') as writer:
  dados_comuns.to_excel(writer, sheet_name='Dados em comum', index=False)
    
with pd.ExcelWriter('dados_left.xlsx') as writer:
  dados_left.to_excel(writer, sheet_name='Dados exclusivos do arquivo A', index=False)
    
with pd.ExcelWriter('dados_right.xlsx') as writer:
  dados_right.to_excel(writer, sheet_name='Dados exclusivos do arquivo B', index=False)
